# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [74]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import string

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, fbeta_score

from sklearn.base import BaseEstimator, TransformerMixin


In [17]:
import nltk

In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [41]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [21]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('labelledresponses', engine)
X = df['message']
Y = df.iloc[:,4:]

df.head(5)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [22]:
stop_words = nltk.corpus.stopwords.words("english")
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
remove_punc_table = str.maketrans('', '', string.punctuation)

In [23]:
def tokenize(text):
    #nomalize
    text = text.translate(remove_punc_table).lower()
    
    #tokenize text
    tokens = nltk.word_tokenize(text)
        
    return [lemmatizer.lemmatize(x) for x in tokens if x not in stop_words]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [24]:
clf_forest = RandomForestClassifier(n_estimators = 10)

pipeline = Pipeline([
                    ('tfidf', TfidfVectorizer()), 
                    ('forest',MultiOutputClassifier(clf_forest))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.30, random_state = 42 )

In [26]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [27]:
y_pred = pd.DataFrame(pipeline.predict(X_test), columns = df.columns[4:])
                      
for i in y_pred.columns:
    print(i,"\n")
    print(classification_report(y_pred[i],Y_test[i]))
    print("-----------------\n")

related 

             precision    recall  f1-score   support

          0       0.34      0.60      0.44      1061
          1       0.93      0.81      0.87      6744
          2       0.55      0.53      0.54        60

avg / total       0.84      0.78      0.81      7865

-----------------

request 

             precision    recall  f1-score   support

          0       0.98      0.89      0.93      7228
          1       0.39      0.81      0.53       637

avg / total       0.93      0.88      0.90      7865

-----------------

offer 

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      7865
          1       0.00      0.00      0.00         0

avg / total       1.00      1.00      1.00      7865

-----------------

aid_related 

             precision    recall  f1-score   support

          0       0.88      0.72      0.79      5630
          1       0.52      0.74      0.61      2235

avg / total       0.77      0.73      0.74

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [28]:
from sklearn.model_selection import GridSearchCV

parameters = {
                'tfidf__ngram_range': ((1,1), (1,2)),
                'tfidf__max_df': (0.8,1.0),
                'tfidf__max_features': (None, 10000),
                'forest__estimator__n_estimators': [50,100,200],
                'forest__estimator__min_samples_split': [2,4]
}

cv = GridSearchCV(pipeline, parameters, cv=3, n_jobs=1)

In [33]:
cv.fit(X_train, Y_train)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [34]:
def evaluate_model(model, X_test, Y_test):
    Y_pred = pd.DataFrame(model.predict(X_test), columns = df.columns[4:])
    
    for i in Y_pred.columns:
        print("-----------------\n")
        print("-----------------\n")
        
        print("Feature: " ,i,"\n")
        print(classification_report(Y_test[i],Y_pred[i]))
        
        print("-----------------\n")
        
        print('Accuracy: ', accuracy_score(Y_test[i],Y_pred[i]))
        print('Precision: ', precision_score(Y_test[i],Y_pred[i], average='weighted'))
        print('Recall: ', recall_score(Y_test[i],Y_pred[i], average='weighted'))
        
        print("-----------------\n")
        print("-----------------\n")
    
    return Y_pred


In [36]:
Y_pred = evaluate_model(pipeline, X_test, Y_test)

-----------------

-----------------

Feature:  related 

             precision    recall  f1-score   support

          0       0.60      0.34      0.44      1873
          1       0.81      0.93      0.87      5934
          2       0.53      0.55      0.54        58

avg / total       0.76      0.78      0.76      7865

-----------------

Accuracy:  0.784106802289
Precision:  0.762332962752
Recall:  0.784106802289
-----------------

-----------------

-----------------

-----------------

Feature:  request 

             precision    recall  f1-score   support

          0       0.89      0.98      0.93      6533
          1       0.81      0.39      0.53      1332

avg / total       0.88      0.88      0.86      7865

-----------------

Accuracy:  0.881627463446
Precision:  0.875197579643
Recall:  0.881627463446
-----------------

-----------------

-----------------

-----------------

Feature:  offer 

             precision    recall  f1-score   support

          0       1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Precision:  0.992892521568
Recall:  0.996439923713
-----------------

-----------------

-----------------

-----------------

Feature:  aid_centers 

             precision    recall  f1-score   support

          0       0.99      1.00      0.99      7762
          1       0.00      0.00      0.00       103

avg / total       0.97      0.99      0.98      7865

-----------------

Accuracy:  0.986904005086
Precision:  0.973979515254
Recall:  0.986904005086
-----------------

-----------------

-----------------

-----------------

Feature:  other_infrastructure 

             precision    recall  f1-score   support

          0       0.96      1.00      0.98      7524
          1       0.00      0.00      0.00       341

avg / total       0.92      0.96      0.94      7865

-----------------

Accuracy:  0.956261919898
Precision:  0.91515068498
Recall:  0.956261919898
-----------------

-----------------

-----------------

-----------------

Feature:  weather_related 

             pr

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [96]:
class first_verb(BaseEstimator, TransformerMixin):
    
    def first_verb(self, message):
        sent_list = nltk.sent_tokenize(message)
        for x in sent_list:
            pos_tags = nltk.pos_tag(tokenize(x))
            if len(pos_tags) >0:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            else:
                False
        return False
    
    def fit(self,X,y=None):
        return self
    
    def transform(self, X):
        X_tag = pd.Series(X).apply(self.first_verb)
        return pd.DataFrame(X_tag)

In [97]:
def model_pipeline():
    pipeline = Pipeline([
        ('features', FeatureUnion([

            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer())
            ])),

            ('first_verb', first_verb())
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
    ])

    return pipeline

In [98]:
def model_performance(Y_test,Y_pred, beta = 0.5):
    '''
    Input: Actual and Predicted Values both as DataFrames for multiclass multioutput problem" 
    Output:Model performance for each class 
    '''
    if isinstance(Y_test, pd.DataFrame) == True &  isinstance(Y_pred, pd.DataFrame) == True:
        metrics = {}
        for col in Y_test.columns:
            f_score   =  fbeta_score(Y_test[col],Y_pred[col],beta ,average='weighted')
            precision =  precision_score(Y_test[col],Y_pred[col] ,average='weighted')
            recall    =  recall_score(Y_test[col],Y_pred[col] ,average='weighted')
            
            metrics[col] = [f_score, precision, recall]
        
        return pd.DataFrame.from_dict(metrics, orient='index', columns=['f_score_'+str(beta), 'precision',  'recall'])

    else:
        return "Check input"
    

In [86]:
x  = model_performance(Y_test,Y_pred, beta = 0.5)

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 9. Export your model as a pickle file

In [99]:
model_new = model_pipeline()
model_new.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [100]:
Y_pred = model_new.predict(X_test)

In [101]:
performance  = model_performance(Y_test,Y_pred, beta = 0.5)

In [ ]:
filename = 'classifier_model.sav'
pickle.dump(model_, open(filename, 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.